In [1]:
import pandas as pd
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
market = Market()
sec = SEC()
fin = ADatabase("financial")
umod = UniversalModeler()

In [3]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [4]:
factors = ['assets',
 'liabilitiesandstockholdersequity',
 'incometaxexpensebenefit',
 'retainedearningsaccumulateddeficit',
 'accumulatedothercomprehensiveincomelossnetoftax',
 'earningspersharebasic',
 'earningspersharediluted',
 'propertyplantandequipmentnet',
 'cashandcashequivalentsatcarryingvalue',
 'entitycommonstocksharesoutstanding',
 'weightedaveragenumberofdilutedsharesoutstanding',
 'weightedaveragenumberofsharesoutstandingbasic',
 'stockholdersequity']

In [5]:
included_columns=[
    "year",
    "quarter",
    "ticker",
    "adjclose",
    "y"
]

In [6]:
included_columns.extend(factors)

In [ ]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        prices = market.retrieve_ticker_prices("prices",ticker)
        prices = p.column_date_processing(prices)
        prices["year"] = [x.year for x in prices["date"]]
        prices["quarter"] = [x.quarter for x in prices["date"]]
        filing = sec.retrieve_filing_data(cik)
        filing = p.column_date_processing(filing)
        filing = filing.groupby(["year","quarter"]).mean().reset_index()
        ticker_data = prices.copy()
        ticker_data.sort_values("date",ascending=True,inplace=True)
        ticker_data["adjclose"] = [float(x) for x in ticker_data["adjclose"]]
        ticker_data = ticker_data.groupby(["year","quarter"]).mean().reset_index()
        ticker_data.dropna(inplace=True)
        ticker_data["ticker"] = ticker
        ticker_data["y"] = ticker_data["adjclose"].shift(-4)
        ticker_data = ticker_data.merge(filing,on=["year","quarter"],how="left").reset_index()
        ticker_data = ticker_data[included_columns]
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

  1%|██                                                                                                                                                                         | 6/490 [00:01<01:56,  4.16it/s]

"['entitycommonstocksharesoutstanding'] not in index"


  2%|██▊                                                                                                                                                                        | 8/490 [00:02<02:33,  3.14it/s]

"['stockholdersequity'] not in index"


  4%|██████▌                                                                                                                                                                   | 19/490 [00:06<02:43,  2.88it/s]

...

"['stockholdersequity'] not in index"


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                | 443/490 [02:41<00:13,  3.58it/s]

"['earningspersharebasic', 'entitycommonstocksharesoutstanding', 'weightedaveragenumberofsharesoutstandingbasic'] not in index"


 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊               | 446/490 [02:43<00:15,  2.77it/s]

"['retainedearningsaccumulateddeficit', 'propertyplantandequipmentnet'] not in index"


In [ ]:
data = pd.concat(training_sets)

In [ ]:
data = data.dropna()

In [ ]:
data

In [ ]:
fin.connect()
years = 4
for modeler in [umod]:
    for year in tqdm(range(2016,2023)):
        training_slice = data[(data["year"]<year) & (data["year"] >= year - years)].reset_index(drop=True)
        prediction_set = data[data["year"]==year].reset_index(drop=True)
        set_name = "sim"
        stuff = modeler.model(training_slice,prediction_set,factors,tf=False)
        stuff["training_years"] = years
        fin.store(set_name,stuff)
fin.disconnect()

In [ ]:
stuff